# Solving the 1 dimensional Schrödinger equation with a matrix method

## Introduction

Quantum mechanics is an important branch in physics that is used to describe physical behaviours at the smallest scales. The description of physical systems depends on solving the  Schrödinger equation since the solutions can be used to describe the behaviour of subatomic, atomic and molecular systems. However, it can be solved analytically for very few systems, requiring a sufficiently good numerical method when no analytical solution exists.  [[1]](https://www.researchgate.net/publication/322593570_A_Matrix_Method_of_Solving_the_Schrodinger_Equation)

Such a numerical algorithm, the matrix method, was used to solve the Schrödinger equation where the boundary conditions are known. A basis set of functions were chosen which obey boundary conditions and linear combinations of them were found in order to approximate the wave function. The wave function was then substituted into the Schrödinger equation and pre-multiplied by each of the basis. By integrating, a series of linear equations was obtained that was written as a matrix equation. The matrix was then represented in terms of its eigenvalues (eigenenergies) and eigenvectors (corresponding multipliers of  basis).[[2]](http://demonstrations.wolfram.com/ExactSolutionForRectangularDoubleWellPotential/)

In this project, the one-dimensional, time-independent Schrödinger equation is solved for two systems: the particle in a box, with and without a potential barrier. The particle in a box problem is a common application of a quantum mechanical model to a simplified system. It describes the horizontal translation motion of a particle confined in an infinitely deep well from which it cannot escape. Adding a potential barrier inside the infinite well allows us to study quantum tunneling; a phenomenon in which a particle penetrates a potential energy barrier with a height greater than the energy of the particle. The phenomenon is exciting because it violates the principles of classical mechanics by allowing particles can get through classically forbidden regions and is possible because in quantum mechanics, particles show wave properties. It is essential in modeling the Sun and other stars, and has a wide range of applications, such as transistors, integrated circuits and the scanning tunneling microscope.[[3]]()

The analytical solution for both systems is known so they were compared to the numerical solutions, testing the suitability of the matrix method. For the infinite square well, the results match the analytical solutions well. However, when the barrier was added, the solutions were correct for a small range of the initial parameters.

## Method

The model of a particle is trapped between two regions of infinite potential forms the one dimensional $\textbf{infinite potential well}$ as shown in Figure 1. 

<img src="pbox.gif" width = '240'> $\textit{Figure 1}. $ Particle is trapped inside an infinite potential well of width $L$. The first three wavefunctions are shown. Adapted from [[4]](http://hyperphysics.phy-astr.gsu.edu/hbase/quantum/pbox.html)

The time-independent Schrödinger equation for this simple system is:
$$
-\dfrac{\hslash^2}{2m} \, \dfrac{\mathrm{d}^2 \psi}{\mathrm{d} x^2} = E\psi  \;\;\;(1.1)
$$
or $$\mathcal{\hat{H}}\psi = E\psi  \;\;\;(1.2)$$
where $\hslash$ is the reduced Plank's constant: $\dfrac{h}{2\pi}$, $m$ is the particle's mass, $E$ the eigen-energies, and $\psi$ the wavefunctions. In this project $L=2$ and natural units are taken so that $\dfrac{\hslash^2}{2m}$ equals 1.

The objective is to chose a suitable basis set of N functions $φ_i(x)$ and  to find linear combinations of them such that:
$$\psi = \sum_i^n c_i \phi_i \;\;\;(2)$$  where $c_i$ are constants to be found.

Then multiply $\psi$ by the complex conjugate $\phi*$ (which is just $\phi$ in this case as the basis chosen are real). Substitute into eq. (1.2) and integrate will result in a series of linear equations which can be represented by the matrix equation:
$\textbf{Ha} = E\textbf{Sa}$, where $\textbf{H}$ and $\textbf{S}$ are matrices, a is an eigenvector and $E$ energy levels.

This is a form of a generalised eigenvalue problem and `scipy.linalg.eig` can be used to solve it. It requires an input of the   matrix whose eigenvalues and eigenvectors will be computed and, in this case, the right-hand side matrix in the problem. The function returns a tuple (eigvals,eigvecs) where eigvals (set of $E$) is a 1D array of numbers, and eigvecs is a 2D array with the corresponding eigenvectors (set of $\textbf{a}$) in the columns.[[5]](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eig.html)




The $\textbf{double-well potential}$ is composed of a finite barrier between the infinite walls of potential. 

<img src="double_well.png" width = '1000'>

$\textit{Figure 2}$. Simplistic rectangular double-well potential showing even and odd eigenstates. 
Adapted from [[6]](https://chemistry.stackexchange.com/questions/85447/does-possibility-of-tunneling-influence-ground-state-energy-of-pib-with-and-with).

Two wavefunctions  are shown in Figure 2. The solutions closely resemble a symmetric and an antisymmetric linear combination of the ground state of a particle in an infinite box. The same basis as the infinite well was tried but resulted in erroneously complex results. Hence, the basis used is that of the exact solutions of the infinite well. However, only the symmetric functions were calculated in this project.

Two functions, one for each system, were created which evaluate the matrix equation using `scipy.linalg.eig`. The functions called two other plotting functions created earlier. The `interact` function from the module `ipywidgets` was used to automatically create user interface controls allowing the variance of certain parameters of the systems (number of basis N, width and height of barrier). Interact autogenerated a slider bound to each parameter, and then called each function with those arguments, allowing them to be manipulated interactively. [[7]](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html)

### Importing relevant libraries and functions

In [1]:
import numpy as np
from scipy.linalg import eig
from matplotlib import pyplot as plt
from ipywidgets import interact

### Basis

As basis functions for the infinite well take simple polynomials that vanish at the walls:

$$\phi_i(x) = x^i(x-1)(x+1), \; i = 0,1,2,...n. \;\;\;(3.1)$$

In [2]:
x = np.linspace(-1,1,int(10e3)) # range of well

In [3]:
def poly_basis(N):
    Φ_functions = []
                    
    for i in range(N+1):
        Φ = x**i * (x - 1)*(x + 1)
        Φ_functions.append(Φ)
                
    return Φ_functions


For the double well, even excited states were evaluated using cosine based functions: 

$$ \phi_i(x) = cos(\frac{ix\pi}{2}), \; i = 0, 2, 4,...n. \;\;\;(3.2)$$

In [4]:
def cos_basis(N):
    Φ_functions = []
                    
    for i in range(N+1):
        Φ =   np.cos(((2*i+1)*x*np.pi)/2)
        Φ_functions.append(Φ)
        
    return Φ_functions

### Building and solving the matrices

In order to solve for $\psi$ and $E$ in eq. (2) the Hamiltonian (energy) matrix $\textbf{H}$ and  the overlap matrix $\textbf{S} $ are constructed.

The reason for choosing the particular form of basis functions is that the relevant matrices' elements can easily be calculated:

$$
S_{ij} = \int_{-1}^{1}\phi_i^*\phi_j dx \approx \; \sum_i^n \sum_j^n\phi_i^* \phi_j \;\;\;\;\;\;\; \;\;\;(4.1)
$$

and the elements of the matrix $\textbf{H}$ are:

$$ 
\begin{equation}\label{Hamiltonian}
H_{ij} = \int_{-1}^{1}\phi_i^* \mathcal{\hat{H}} \phi_jdx \approx \sum_i^n \sum_j^n\phi_i^* \mathcal{\hat{H}} \phi_j \;  \; \; \;\;\;(4.2)
\end{equation}.
$$
$\mathcal{\hat{H}} = -\dfrac{d^2}{dx^2}$ for the infinite square well and
$ \mathcal{\hat{H}} =-\dfrac{d^2}{dx^2} + V(x) $  for the double well, where [V(x)](#Simple-double-well). is a potential barrier of varying height and width: $$ V(x) =\begin{gather*}    
\begin{cases}
  V = \text{height} & \text{for }|x|\le \frac{\text{width}}{2}\\    
  V = 0  &\text{elsewhere   }
\end{cases}
\end{gather*} \;\;\;(5).$$                                             
                                                                                                  

In [5]:
def eigen_states(basis_functions,potential=None):
    """Returns sets of sorted eigen-energies and constants of an infinite potential well and a given potential.
    
    Parameters 
    ----------
    basis_functions: array
        Chosen estimates of basis functions.
    potential: function, optional
        An additional function to be added between the walls of the well.
        
    Returns
    -------
    sorted_eigenvectors: ndarray
        Eigenvector of system ordered in terms of ascending eigenvalues.
    sorted_eigenvalues: array
        Eigenvalue of system in ascending order.
    """
    
    n = len(basis_functions)
    
    S = np.zeros((n,n))   # S matrix
    H = np.zeros((n,n))  # Hameltonian matrix
    
    for i in range(n):
        for j in range(n):
            S[i,j] = np.trapz( basis_functions[i] * basis_functions[j]) # Consructing eq. (4.2)
            
            diff2 = -np.gradient ( np.gradient( basis_functions[j],x ),x ) # second differential of basis function
            if potential:
                diff2 += potential(x)
                
            H[i,j] = np.trapz( basis_functions[i] * diff2) # Constructing eq. (4.1)

            
    eigvals, eigvecs = eig(H, S) # Solving generalised eigenvalue problem using scipy.linalg.eig
    
    # Sorting in ascending magnitude of eigenvalues
    
    eigvals_indices = eigvals.argsort()
    sorted_eigvals = eigvals[eigvals_indices]
    sorted_eigvecs = eigvecs[:,eigvals_indices]
    
    return (sorted_eigvecs, sorted_eigvals)

### Plotting functions

In [6]:
def well_plot(title, d=0 ):
    """Creates a blank graph for wavefunctions to be plotted
    
    Parameters 
    ----------
    title: string
        name of well, i.e. 'infinite square' or 'double'
    d: float
        number for arrow and infinity symbol to be shifted by from 6
        
    Returns
    -------
    Labeled figure of with axes and arrows at x=-1,1
    sorted_eigenvectors: ndarray
    """
    ax = plt.axes((1,1,1.5,1.2)) # create axes
    
    #Add arrows at walls of well with infinity symbols
    ax.arrow(-1, 0, 0, 5.8 + d, head_width=0.05, head_length=0.1, fc='k', ec='k')
    ax.arrow(1, 0, 0, 5.8 + d, head_width=0.05, head_length=0.1, fc='k', ec='k')
    
    ax.annotate(('$\infty$'),
            xy=(1, 1),
            xytext=(-1.03, 6 + d),
            size = 20)
        
    ax.annotate(('$\infty$'),
            xy=(1, 1),
            xytext=(0.97, 6 + d),
            size = 20)
    
    plt.xlabel('x')
    plt.ylabel('Excited state')
    plt.yticks(np.linspace(0,6,7))
    plt.xticks(np.linspace(-1,1,3))
    plt.ylim((0,6.3))
    plt.title('Wavefunctions of ' + title + ' well')
    plt.grid()
    
    
    
def energy_plot(title):
    """Creates blank graph for energies to be plotted
   
    Parameters 
    ----------
    title: string
        name of well, i.e. 'infinite square' or 'double'
        
    Returns
    -------
    Labeled figure with axes
    """
    plt.figure(figsize=(11.7,6))
    plt.title('Energy eigenstates of ' + title + ' well')
    plt.xlabel('Energy level (n)')
    plt.ylabel('Energy')
    plt.grid()

### Calculating wavefunctions and energies

### Infinite square well

In [7]:
def infinite_well(N):
    """Plots wavefunctions and energies of infinite square well with varying number N of polynomial bases
    
    Parameters 
    ----------
    N: integer
         number of polynomial functions to be used
        
    Returns
    -------
    Plots first 5 wavefunctions and corresponding exact and calculated eigen-energies of system
    """
    
    well_plot('infinite square')
    
    Φ_functions = poly_basis(N)
    eigvecs, eigvals = eigen_states(Φ_functions)
    
    # Multiplying the eigenvectors (c_i) with the basis functions
    for En in range(5):
        y = np.zeros_like(x)
        for i in range(N):
            y += eigvecs[i,En] * Φ_functions[i] 
        plt.plot(x,y.real*(En**2+1) + En+1) # multiplied by an increasing value so as to make waves visible
        
    exact_energies = [((n+1) *np.pi)**2 /4 for n in range(N+1)] # using eq. 6 to calculate exact eigen-energies
    energy_plot('infinite square')
    plt.plot(exact_energies,'o-',label='Exact energies')
    plt.loglog(eigvals.real,'o',label='Calculated energies')
    plt.legend()

### Simple double-well

In [8]:
def double_well(Height, Width, N):   
    """Plots wavefunctions and of double well with varying number of basis N, height and width of barrier
        
    Parameters 
    ----------
    N: integer
         number of cosine bases to be used
    Height: integer
        height of barrier (potential) inside well
    Width: float
        width of barrier 
        
    Returns
    -------
    Plots first 3 even wavefunctions and corresponding calculated eigen-energies of system
    """
    
    def potential_barrier(x):     # V(x)
        # Constructing potential as in the function (5)
        y = np.zeros(x.shape)
        y[abs(x) < Width/2] = Height
        return y

    well_plot('double', 0.8)
    plt.ylim((1,7))
    
    Φ_functions = cos_basis(N)
    eigvecs, eigvals = eigen_states(Φ_functions, potential_barrier)
    
    if np.iscomplexobj(eigvecs)==False: # check if eigenvectors are complex, and plot if they are real
    
        for En in range(3):
            y = np.zeros_like(x)
            for i in range(N):
                y += eigvecs[i,En] * Φ_functions[i]
            plt.plot(x,y.real + 2*(En +1))
            plt.text(1, 2*(En+1), "{0:.2f}".format(eigvals[En].real)) # Displaying energy of each plotted wave
            
        levels = [n for n in range(2*N+2) if n%2==0] # Even energy levels
        energy_plot('double')
        plt.loglog(levels,eigvals.real, 'o')
        
    else:
        plt.close()
        print('Error: Eigenvectors are complex.\n')

## Results

### Infinite square well 

In [9]:
interact(infinite_well, N = (5, 14, 1))    

interactive(children=(IntSlider(value=9, description='N', max=14, min=5), Output()), _dom_classes=('widget-int…

<function __main__.infinite_well(N)>

$\textit{Figure}\; \textbf{3}\$ a) Wavefunctions of the first 5 states and b) calculated and exact energies of an infinite quantum well. The wavefunctions have an arbitrary magnitude and are shifted by the corresponding electron energy.

The magnitude of the wavefunctions decreased with each level so they had to be multiplied by $(En^2+1)$ in order for the higher states to be clearly visible. 

The number of bases used has a great influence on the shape of the wavefunctions. A greater number of bases is needed to evaluate more energetic particles. Even with N=5, the first 3 waves seem reasonable. The minimum of polynomials needed to obtain the two highest wavefunctions was 9, although at $N$=10, there seems to be an anomaly with the 4$^{\text{th}}$ wave. The sign of the wave appears to alternate with each additional basis. 

Although the energy graphs are both exponential, the calculated energies rise faster than the exact energies: $$E_e = \dfrac{\hslash}{2m}\dfrac{(n\pi)^2}{4} = \dfrac{(n\pi)^2}{4} \;\;\;(6).$$ 

#### Speed of algorithms

In [10]:
plt.rcParams.update({'figure.max_open_warning': 0}) #surpress warnings

In [11]:
%timeit eigen_states(poly_basis(10))

59.2 ms ± 451 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%timeit eigen_states(poly_basis(20))

226 ms ± 13.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit infinite_well(10)
plt.close('all')

111 ms ± 22.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Double-well:  symmetric solutions

In [14]:
interact(double_well, Height = (80, 95, 1), Width = (0.25, 0.6, 0.05), N = (3, 10, 1) )  

interactive(children=(IntSlider(value=87, description='Height', max=95, min=80), FloatSlider(value=0.4, descri…

<function __main__.double_well(Height, Width, N)>

$\textit{Figure } \textbf{4}$ a) The first 3 symmetrical wavefunctions and (b) N energy states in an simple double quantum well. The wavefunctions are not normalized and are shifted by the corresponding energy level. 

The eigenvectors are very sensitive to all the parameters. A correct solution is only possible for a small number of combinations of N, height and width. The eigenvectors always become complex when the width is greater or equal to 0.50.

The magnitude of the waves decreases only slightly with level, possibly because the cosine basis is a better approximation than the polynomial basis. The waves also seem to be inverted depending on N, as with the infinite well. For example, the third wavefunction appears to erroneously rise at the barrier at [Height = 94, Width = 0.25, N = 6]. The energy of the particles is lower than the barrier (i.e. $E\le V$), so it should be lower.

Eigen-energies correctly increase with the width of the barrier.




### Speed of algorithms

In [15]:
%timeit eigen_states(cos_basis(10)) 

60.3 ms ± 4.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit eigen_states(cos_basis(20))

207 ms ± 6.38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit double_well(90, 0.3, 10)
plt.close('all')

92.6 ms ± 5.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)



## Discussion

The graphs rightly show that in that the lowest possible energy of a particle is never zero even when the potential is zero within the well. This suggests the particle can never be at rest and will always have some kinetic energy, even at zero Kelvin. This obeys the Heisenberg Uncertainty Principle: if a particle's energy was zero, its position in time would be precisely known. The energies rightly increase logarithmically. However, the higher the energy the more inaccurate it is and always higher than the actual value.

Few basis functions are needed to obtain accurate wavefunctions if the basis functions and exact solutions are very similar. At least 9 polynomial functions were needed, more than the cosine functions (5) to yield the correct wavefunctions. For both systems, an increase in basis improves the wavefunctions but worsens the energies.The inversion of wavefunctions may be due to a phase change rather than multiplication by a negative sign as it depends on N. The erroneous highest wave of the double well indicates that only one of the phases can be correct.

The algorithms are not very fast and become slower with N. A large part of the time taken is in calculating the eigenvalues.


## Conclusions


The infinite well is the simplest system and was evaluable. However, by adding a barrier, it was impossible to obtain any results unless the exact solutions were used as ansatz. If the basis chosen is suitable, for more accurate results simply use a greater number of basis. However, when tested on such a basic system as the simple double-well, a problem of inversion of the wavefunctions occurs. This should be fixed before applying the algorithm to any more complicated systems of which the solutions are not known. The algorithm also appears to be limited, as it cannot produce even an approximate solution for certain parameters, such for the double-well when Width$ \geq $0.50 units. 

Overall the matrix method is successful in some respects and is a relatively simple and fast. Having bigger matrices and N or plotting more of the wavefunctions will likely slow it down considerably. The time taken for just producing the eigenvalues for the polynomials, more than doubled at one instance from 180 ± 22.1 ms to 522 ± 13.5 ms per loop when doubling N.

The method could be useful when the exact solutions are not known, but the basis functions must be very good ansatz. It could also be easily adapted to work with complex basis by taking the complex conjugate of the first ($i^\text{th}$) basis when creating the matrices. For a more quantitative analysis the basis can be normalised and a curve can be fitted to the energy values, so that both could be compared with the analytical solutions. 

## References

[1] Okock P. O., Burns T. $\textit{A Matrix Method of Solving the Schrodinger Equation}$, (Tanzania, AIMS 2015).

[2] Blinder S. M., $\textit{Exact Solution for Rectangular Double-Well Potential}$ Available from: .
http://demonstrations.wolfram.com/ExactSolutionForRectangularDoubleWellPotential/
Wolfram Demonstrations Project (2013). [Accessed 19 Dec. 2018]

[3] Chang R., $\textit{Physical Chemistry for the Biosciences}$. (Sansalito, CA: University Science, 2005).

[4] Nave R. (2000)  $\textit{Particle in a box}$ [Image] Available from: http://hyperphysics.phy-astr.gsu.edu/hbase/quantum/pbox.html [Accessed 18 Dec. 2018]

[5] scipy.linalg.eig — SciPy v1.1.0 Reference Guide [Internet]. Docs.scipy.org. 2018. Available from: https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eig.html [Accessed 20 Dec. 2018]

[6] Chemistry Stack Exchange (2018) [Image]. Available from: https://chemistry.stackexchange.com/questions/85447/does-possibility-of-tunneling-influence-ground-state-energy-of-pib-with-and-with [Accessed 23 Dec. 2018].

[7] ipywidgets.interact - Jupyter Widgets[Internet] Available from: https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html [Accessed 3 Jan. 2019]